### Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import load_model
import os

### Data loading

In [2]:
base_dir="dataset"

In [3]:
class_labels=os.listdir(base_dir)
print(class_labels)

['Acinetobacter.baumanii', 'Actinetobacter.israeli', 'Candida.albicans', 'Clostridium.perfringens', 'Fusobacterium', 'Lactobacillus.casei', 'Lactobacillus.paracasei', 'Listeria.monocytogenes', 'Porfyromonas.gingivalis', 'Veionella']


In [4]:
data=[]
count=0
for label in class_labels:
    print(f"loading data from {os.path.join(base_dir,label)}")
    files=os.listdir(os.path.join(base_dir,label))
    for file_ in files:
        fullpath=os.path.join(base_dir,label,file_)
        try:
            image=load_img(path=fullpath,
                    grayscale=False,
                    color_mode='rgb',
                    target_size=(128,128))
            array_image=img_to_array(image)
            normalized_image=array_image/255.0
            data.append([normalized_image,count])
        except Exception as e:
            continue
    count+=1
print("Data loading completed")

loading data from dataset\Acinetobacter.baumanii
loading data from dataset\Actinetobacter.israeli
loading data from dataset\Candida.albicans
loading data from dataset\Clostridium.perfringens
loading data from dataset\Fusobacterium
loading data from dataset\Lactobacillus.casei
loading data from dataset\Lactobacillus.paracasei
loading data from dataset\Listeria.monocytogenes
loading data from dataset\Porfyromonas.gingivalis
loading data from dataset\Veionella
Data loading completed


### Data preprocessing

In [5]:
X,y=zip(*data)

In [6]:
X=np.asarray(X)
y=np.asarray(y)
X,y=shuffle(X, y, random_state=42)
print(X.shape,y.shape)

(11184, 128, 128, 3) (11184,)


### Model loading

In [7]:
cnn_model=load_model("model/ConvolutionalNeuralNetwork_model.h5")

In [8]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 128)      512       
 hNormalization)                                                 
                                                        

### Feature extraction

In [9]:
from tensorflow.keras.models import Model
intermediate_layer_model=Model(inputs=cnn_model.input,
                               outputs=cnn_model.get_layer('dense_2').output)

In [11]:
image_features=intermediate_layer_model.predict(X,batch_size=32,verbose=1)

350/350 [==============================] - 288s 822ms/step


### Extracted features in tabular format

In [12]:
column_names=[]
for i in range(128):
    column_names.append("feature_"+str(i+1))

features_df=pd.DataFrame(data=image_features,columns=column_names)
features_df['label']=y
features_df.head(10)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,label
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,10.819415,0.0,0.0,0.0,7
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,10.390684,0.0,0.0,0.0,7
3,0.0,0.0,26.509972,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,4
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,52.810894,0.0,0.0,0.0,3
5,0.0,0.0,0.000000,0.0,0.0,0.0,12.951874,0.0,0.0,1.918265,...,0.0,0.0,0.0,0.0,0.0,18.786961,0.0,0.0,0.0,8
6,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,33.639439,0.0,0.0,0.0,3
7,0.0,0.0,0.000000,0.0,0.0,0.0,13.505110,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
8,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,10.609745,0.0,0.0,0.0,7
9,0.0,0.0,0.000000,0.0,0.0,0.0,34.417782,0.0,0.0,18.280046,...,0.0,0.0,0.0,0.0,0.0,42.696049,0.0,0.0,0.0,8


### saving the data

In [13]:
features_df.to_csv("input/images_features.csv",index=False)